## Zadanie domowe - Algorytm Canny'ego

Celem zadania domowego jest wykonanie pełnej implementacji algorytmu Canny'ego.

W ramach ćwiczenia w trakcie laboratorium wyznaczono obrazy $g_{NH}$ i $g_{NL}$.
Dla przypomnienia:
Można powiedzieć, że na obrazie $g_{NH}$ są "pewne" krawędzie.
Natomiast na $g_{NL}$ "potencjalne".
Często krawędzie "pewne" nie są ciągłe.
Wykorzystuje się więc krawędzie "potencjalne", aby uzupełnić nieciągłości.
Procedura wygląda następująco:
1. Stwórz stos zawierający wszystkie piksele zaznaczone na obrazie $g_{NH}$.
W tym celu wykorzystaj listę współrzędnych `[row, col]`.
Do pobrania elementu z początku służy metoda `list.pop()`.
Do dodania elementu na koniec listy służy metoda `list.append(new)`.
2. Stwórz obraz, który będzie zawierał informację czy dany piksel został już odwiedzony.
3. Stwórz obraz, który zawierać będzie wynikowe krawędzie.
Jej rozmiar jest równy rozmiarowi obrazu.
4. Wykonaj pętlę, która będzie pobierać elementy z listy, dopóki ta nie będzie pusta.
W tym celu najlepiej sprawdzi się pętla `while`.
    - W każdej iteracji pobierz element ze stosu.
    - Sprawdź, czy dany element został już odwiedzony.
    - Jeśli nie został, to:
        - Oznacz go jako odwiedzony,
        - Oznacz piksel jako krawędź w wyniku,
        - Sprawdź otoczenie piksela w obrazie $g_{NL}$,
        - Dodaj do stosu współrzędne otoczenia, które zawierają krawędź.
        Można to wykonać np. pętlą po stworzonym otoczeniu.
7. Wyświetl obraz oryginalny, obraz $g_{NH}$ oraz obraz wynikowy.
8. Porównaj wynik algorytmu z wynikiem OpenCV.

Pomocnicze obrazy $g_{NH}$ i $g_{NL}$ zostały wprowadzone dla uproszczenia opisu.
Algorytm można zaimplementować w bardziej "zwarty" sposób.

Na podstawie powyższego opisu zaimplementuj pełny algorytm Canny'ego.

In [ ]:
import cv2
from matplotlib import pyplot as plt
import numpy as np
import math
import os

if not os.path.exists("dom.png") :
    !wget https://raw.githubusercontent.com/vision-agh/poc_sw/master/09_Canny/dom.png --no-check-certificate

I_house = cv2.imread('dom.png',  cv2.IMREAD_GRAYSCALE)

def nonmax(destination_matrix, amplitude):
    X,Y = destination_matrix.shape
    gn = np.zeros((X,Y))
    
    for i in range(1, X-1):
        for j in range(1, Y-1):
            if destination_matrix[i, j] == 1:
                if amplitude[i, j - 1] > amplitude[i, j] or amplitude[i, j + 1] > amplitude[i, j]:
                    gn[i, j] = 0
                else:
                    gn[i, j] = amplitude[i, j]
            elif destination_matrix[i, j] == 2:
                if amplitude[i - 1, j] > amplitude[i, j] or amplitude[i + 1, j] > amplitude[i, j]:
                    gn[i, j] = 0
                else:
                    gn[i, j] = amplitude[i, j]
            elif destination_matrix[i, j] == 3:
                if amplitude[i - 1, j + 1] > amplitude[i, j] or amplitude[i + 1, j - 1] > amplitude[i, j]:
                    gn[i, j] = 0
                else:
                    gn[i, j] = amplitude[i, j]
            elif destination_matrix[i, j] == 4:
                if amplitude[i + 1, j + 1] > amplitude[i, j] or amplitude[i - 1, j - 1] > amplitude[i, j]:
                    gn[i, j] = 0
                else:
                    gn[i, j] = amplitude[i, j]

    return gn

def canny(image, th, tl):
    X, Y = image.shape
    image_gauss = cv2.GaussianBlur(I_house, (5, 5), 0)
    S1 = np.array([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]])
    S2 = np.array([[-1, -2, -1], [0, 0, 0], [1, 2, 1]])

    # S1 = S1/sum(sum(abs(S1)))
    # S2 = S2/sum(sum(abs(S2)))
    Gx = cv2.filter2D(image_gauss, -1, S1)
    Gy = cv2.filter2D(image_gauss, -1, S2)
    Amplitude = np.sqrt(Gx**2 + Gy**2)
    alfa = np.arctan2(Gy, Gx)
    alfa_deg = alfa*180./np.pi
    alfa_deg[alfa_deg < 0] += 180
    destination_matrix = np.zeros((X, Y))

    for i in range(X):
        for j in range(Y):
            if (0 <= alfa[i,j] < 22.5) or (157.5 <= alfa[i,j] <= 180):
                destination_matrix[i,j] = 1
            elif (22.5 <= alfa[i,j] < 67.5):
                destination_matrix[i,j] = 2
            elif (67.5 <= alfa[i,j] < 112.5):
                destination_matrix[i,j] = 3
            elif (112.5 <= alfa[i,j] < 157.5):
                destination_matrix[i,j] = 4
    
    gn = nonmax(destination_matrix, Amplitude)
    gnh = gn > th
    gnl = np.where(np.logical_and(th>gn,gn>=tl),1,0)
    return gnh, gnl


def canny_2(gnh, gnl, image):
    X, Y = image.shape
    list = []
    for i in range(1, X-1):
        for j in range(1, Y-1):
            if(gnh[i,j]==1):
                list.append([i,j])

    visited = np.zeros((X,Y))
    edges  = np.zeros((X,Y))

    while list:
        p = list.pop()
        x = p[0]
        y = p[1]

        if visited[x, y] == 0:
            visited[x, y] = 1
            edges[x, y] = 1

            for k in range(x-1, x+2):
                for m in range(y-1, y+2):
                    if gnl[k, m] == 1:
                        list.append([k, m])
    return edges

gnh, gnl = canny(I_house, 6, 2)
img_canny = canny_2(gnh, gnl, I_house)
img_canny_builtin = cv2.Canny(I_house, 5, 10, None, 3, 1)

f, (ax1,ax2) = plt.subplots(2,2,figsize=(16,16))
ax1[0].set_title("Oryginał")
ax1[0].imshow(I_house, 'gray')
ax1[0].axis('off')
ax1[1].set_title("Obraz GNH")
ax1[1].imshow(gnh, 'gray')
ax1[1].axis('off')
ax2[0].set_title("Canny napisana własnoręcznie")
ax2[0].imshow(img_canny, 'gray')
ax2[0].axis('off')
ax2[1].set_title("Canny biblioteki")
ax2[1].imshow(img_canny_builtin, 'gray')
ax2[1].axis('off')